In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Install required packages
!pip install -q groq

# Imports
from kaggle_secrets import UserSecretsClient
import groq
from gensim.models import KeyedVectors
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
import os

# === Load Word2Vec Model ===
def load_word_vectors():
    model_path = "/kaggle/input/google-word2vec/GoogleNews-vectors-negative300.bin"
    model = KeyedVectors.load_word2vec_format(model_path, binary=True)
    print("word2vec loaded successfully!")
    return model

model = load_word_vectors()

# === Get Similar Words ===
def get_similar_words(word, model, topn=1):
    try:
        return [w[0] for w in model.most_similar(word, topn=topn)]
    except KeyError:
        return []

# === Groq API Call ===
def generate_response(prompt, model_name="llama3-70b-8192"):
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GROQ_API_KEY")
    if not api_key:
        raise ValueError("Missing GROQ_API_KEY secret.")
    
    client = groq.Client(api_key=api_key)
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# === Enrich Prompt Using Word2Vec ===
def enrich_prompt(prompt, model, max_enrichments=2):
    enriched = []
    for word in prompt.split():
        similar = [w for w in get_similar_words(word, model, topn=max_enrichments) if w.isalpha()]
        enriched.append(f"{word} ({', '.join(similar)})" if similar else word)
    return " ".join(enriched)

# === Compare Responses ===
def analyze_responses(original_response, enriched_response):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([original_response, enriched_response])
    score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    print("\n==== Response Analysis ====")
    print(f"Similarity Score: {score:.4f}")
    print(f"Original Word Count: {len(original_response.split())}")
    print(f"Enriched Word Count: {len(enriched_response.split())}")

# === RUN PIPELINE ===
original_prompt = "Describe the future of artificial intelligence in healthcare in 2 concise bullet points."
enriched = enrich_prompt(original_prompt, model)

# Output prompts
print(f"Original Prompt:\n{original_prompt}")
print(f"\nEnriched Prompt:\n{enriched}")

# Generate and print responses
original_response = generate_response(original_prompt)
enriched_response = generate_response(enriched)

print(f"\n==== Original Response ====\n{original_response}")
print(f"\n==== Enriched Response ====\n{enriched_response}")

# Analyze responses
analyze_responses(original_response, enriched_response)


word2vec loaded successfully!
Original Prompt:
Describe the future of artificial intelligence in healthcare in 2 concise bullet points.

Enriched Prompt:
Describe (Explain) the (this, in) future (furture) of artificial (artifical, Artificial) intelligence (Intelligence, intel) in (inthe, where) healthcare in (inthe, where) 2 concise (succinct, informative) bullet (bullets) points.

==== Original Response ====
Here are 2 concise bullet points that describe the future of artificial intelligence in healthcare:

• **Personalized Medicine**: AI will analyze genomic data, medical records, and real-time health metrics to create tailored treatment protocols for individual patients, leading to more effective disease prevention, diagnosis, and treatment. AI-driven precision medicine will become the standard of care, improving patient outcomes and reducing healthcare costs.

• **Augmented Clinical Decision-Making**: AI-powered tools will assist healthcare professionals in diagnosing and treating 